In [ ]:
!apt-get install
!pip install torchattacks

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
0 upgraded, 0 newly installed, 0 to remove and 38 not upgraded.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.1/50.1 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.0/142.0 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.2/61.2 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 178.7/178.7 kB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.2/144.2 kB 11.1 MB/s eta 0:00:00
  Attempting uninstall: urllib3
    Found existing installation: urllib3 2.5.0
    Uninstalling urllib3-2.5.0:
      Successfully uninstalled urllib3-2.5.0
  Attempting uninstall: idna
    Found existing installation: idna 3.11
    Uninstalling idna-3.11:
      Successfully uninstalled idna-3.11
  Attempting uninstall: chardet
    Found existing installation:

In [ ]:
!pip install wandb

In [ ]:
!pip install wandb

In [ ]:
from __future__ import print_function
import argparse
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.autograd import Variable
import torchvision
import matplotlib.pyplot as plt
import numpy as np
import random
import torchattacks


In [ ]:
x = np.array([[random.randint(0, 1) for _ in range(100)] for _ in range(100)])
x

array([[1, 0, 1, ..., 1, 1, 1],
       [0, 1, 0, ..., 1, 1, 1],
       [0, 1, 1, ..., 0, 1, 0],
       ...,
       [1, 0, 1, ..., 1, 1, 1],
       [1, 0, 1, ..., 0, 1, 1],
       [1, 1, 0, ..., 0, 1, 1]])

In [ ]:
y = []
for i in (range(x.shape[0])):
  if(x[i].sum() % 2 != 0): # Check if the sum is odd
    y.append(1)
  else: # If not odd, it's even
    y.append(0)
y = np.array(y)

In [ ]:
y

array([1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0,
       0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0,
       1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1,
       1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0,
       1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0])

In [ ]:
dimensions = [90, 80, 70, 60, 50, 40, 30, 20, 10, 5]

for n_components in dimensions:
    print(f"\n--- Running with PCA dimension: {n_components} ---")

    # Apply PCA for dimensionality reduction
    pca = PCA(n_components=n_components)
    x_pca = pca.fit_transform(x)

    # Perform KMeans clustering on the PCA-transformed data
    kmeans = KMeans(n_clusters=2, random_state=0, n_init=10)
    clusters = kmeans.fit_predict(x_pca)

    # Count the number of points in each cluster
    cluster_sizes = np.bincount(clusters)
    print("Cluster sizes:")
    for i, size in enumerate(cluster_sizes):
        print(f"  Cluster {i}: {size} points")

    # Calculate the sum of y labels for each cluster and divide by cluster size
    cluster_losses = {}
    for cluster_label in np.unique(clusters):
        # Get the indices of points in the current cluster
        indices = np.where(clusters == cluster_label)[0]

        # Get the corresponding y values
        y_in_cluster = y[indices]

        # Calculate the sum of y values and the cluster size
        sum_y = np.sum(y_in_cluster)
        cluster_size = len(y_in_cluster)

        # Calculate the loss (sum of y / cluster size)
        loss = sum_y / cluster_size if cluster_size > 0 else 0
        cluster_losses[cluster_label] = loss

    # Print the loss for each cluster
    print("Cluster losses:")
    for cluster_label, loss in cluster_losses.items():
        print(f"  Cluster {cluster_label}: Loss = {loss:.4f}")


--- Running with PCA dimension: 90 ---
Cluster sizes:
  Cluster 0: 47 points
  Cluster 1: 53 points
Cluster losses:
  Cluster 0: Loss = 0.3404
  Cluster 1: Loss = 0.4151

--- Running with PCA dimension: 80 ---
Cluster sizes:
  Cluster 0: 56 points
  Cluster 1: 44 points
Cluster losses:
  Cluster 0: Loss = 0.3571
  Cluster 1: Loss = 0.4091

--- Running with PCA dimension: 70 ---
Cluster sizes:
  Cluster 0: 47 points
  Cluster 1: 53 points
Cluster losses:
  Cluster 0: Loss = 0.3404
  Cluster 1: Loss = 0.4151

--- Running with PCA dimension: 60 ---
Cluster sizes:
  Cluster 0: 47 points
  Cluster 1: 53 points
Cluster losses:
  Cluster 0: Loss = 0.3617
  Cluster 1: Loss = 0.3962

--- Running with PCA dimension: 50 ---
Cluster sizes:
  Cluster 0: 45 points
  Cluster 1: 55 points
Cluster losses:
  Cluster 0: Loss = 0.3778
  Cluster 1: Loss = 0.3818

--- Running with PCA dimension: 40 ---
Cluster sizes:
  Cluster 0: 55 points
  Cluster 1: 45 points
Cluster losses:
  Cluster 0: Loss = 0.3636
 

In [ ]:
# Calculate the sum of y labels for each cluster and divide by cluster size
cluster_losses = {}
for cluster_label in np.unique(clusters):
    # Get the indices of points in the current cluster
    indices = np.where(clusters == cluster_label)[0]

    # Get the corresponding y values
    y_in_cluster = y[indices]

    # Calculate the sum of y values and the cluster size
    sum_y = np.sum(y_in_cluster)
    cluster_size = len(y_in_cluster)

    # Calculate the loss (sum of y / cluster size)
    loss = sum_y / cluster_size if cluster_size > 0 else 0
    cluster_losses[cluster_label] = loss

# Print the loss for each cluster
for cluster_label, loss in cluster_losses.items():
    print(f"Cluster {cluster_label}: Loss = {loss:.4f}")

Cluster 0: Loss = 0.2143
Cluster 1: Loss = 0.6591
